In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/
# 查看个人持久化工作区文件
!ls /home/kesci/work/
# 查看当前kernerl下的package
# !pip list --format=columns

In [ ]:
# 下载提交工具至当前目录，仅需执行一次
#!wget -nv -O kesci_submit https://cdn.kesci.com/submit_tool/v1/kesci_submit&&chmod +x kesci_submit
# 提交文件myresult.json进行评审,注意，提交的文件必须为json格式
# !./kesci_submit -token e0abe3caa44ed5b4 -file result.json

In [ ]:
# 查看BROAD数据集目录
!ls /mnt/BROAD-datasets/video/
# 查看文件个数,以training data举例
!ls /mnt/BROAD-datasets/video/training |wc -l
# 查看meta.json
#!cat /mnt/BROAD-datasets/video/meta.json

# 可以看到meta.json有1241个视频，而初赛只有124个视频。所以对meta.json提取：
BROAD_training_filepath="/mnt/BROAD-datasets/video/training"
BROAD_testing_filepath="/mnt/BROAD-datasets/video/testing"
BROAD_validation_filepath="/mnt/BROAD-datasets/video/validation"
# !ls /mnt/BROAD-datasets/video/training

# 以查看get_detection_performance.py进行说明
# !cat /mnt/BROAD-datasets/video/eval_script/get_detection_performance.py

In [ ]:
# 可能导致不输出
# import sys
# reload(sys)
# sys.setdefaultencoding("utf-8")
# %matplotlib inline 

In [ ]:
import json
f=file("/mnt/BROAD-datasets/video/meta.json");
mjson=json.load(f)
f.close()

In [ ]:
# 预览一下meta数据格式。
step=0
database=[]
for d,x in mjson["database"].items():
    step=step+1
    database.append(x)
    if step>=2:
        break
print(json.dumps(database))

In [ ]:
import os ,cPickle,pprint
import pandas as pd

def getLabelArr(video_name_no_behfix):
    labelArr=[]
    for video_name in os.listdir(BROAD_training_filepath):
        video_name_no_behfix=video_name[:-4]
        has_video=mjson["database"].has_key(video_name_no_behfix)
        if(has_video):
    #         构造训练集和标签
            video_info=database[video_name_no_behfix]
            with open(BROAD_training_filepath+"/"+video_name,'rb') as f:
                img_fea=cPickle.load(f)
                len_image=len(img_fea)
                print(img_fea.shape)
    #             pprint.pprint(tmp2)
    #             print(tmp2[1][1])
               # array init
                labelArr = np.zeros([len_image])
                segment_info=video_info['annotations']

                # add seg flag
                for seg_info in segment_info:
                    begin = float(seg_info["segment"][0])
                    begin = math.ceil(begin)
                    begin = int(begin)

                    end = float(seg_info["segment"][1])
                    end = math.floor(end)
                    end = int(end)
                    labelArr[begin:end+1] = 1
        else:
            print(list,"空")
    return labelArr

In [ ]:
import json
import matplotlib.pyplot as plt
with open("/mnt/BROAD-datasets/video/meta.json") as fobj:
    mjson = json.load(fobj)

database = data["database"]
version = data["version"]

def get_sample_frame_from_video(videoid, duration, start_time, end_time,
                                video_path=VIDEO_PATH):
    filename = glob.glob(os.path.join(video_path, "v_%s*" % videoid))[0]
    nr_frames = get_video_number_of_frames(filename)
    fps = (nr_frames*1.0)/duration
    start_frame, end_frame = int(start_time*fps), int(end_time*fps)
    frame_idx = random.choice(range(start_frame, end_frame))
    cap = cv2.VideoCapture(filename)
    keepdoing, cnt = True, 1
    while keepdoing:
        ret, img = cap.read()
        if cnt==frame_idx:
            break
        assert ret==True, "Ended video and frame not selected."
        cnt+=1
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def get_random_video_from_activity(database, activity, subset="validation"):
    videos = []
    for x in database:
        if database[x]["subset"] != subset: continue
        xx = random.choice(database[x]["annotations"])
        if xx["label"]==activity:
            yy = {"videoid": x, "duration": database[x]["duration"],
                  "start_time": xx["segment"][0], "end_time": xx["segment"][1]}
            videos.append(yy)
    return random.choice(videos)

def plotSampleImages():
    plt.figure(num=None, figsize=(18, 50), dpi=100)
    idx = 1
    for ll in leaf_nodes:
        activity = ll["nodeName"]
        keepdoing = True
        while keepdoing:
            try:
                video = get_random_video_from_activity(database, activity)
                img = get_sample_frame_from_video(**video)
                keepdoing = False
            except:
                keepdoing = True
        plt.subplot(20,5,idx)
        idx+=1
        plt.imshow(img), plt.axis("off"), plt.title("%s" % activity)
    plt.show()
plotSampleImages()

In [ ]:
def plotWonderfulTime():
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    import math
    step=0
    wonderfulTimeArr=[]
    for d,x in mjson["database"].items():
        segment_info=x['annotations']
        for seg_info in segment_info:
            begin = float(seg_info["segment"][0])/60.0
            end = float(seg_info["segment"][1])/60.0
            wonderfulTimeArr.append(begin)
            wonderfulTimeArr.append(end)
#         step=step+1
#         if ste/p>=2:
#             break
#     total_duration = sum(wonderfulTimeArr)
#     print("总共精彩时间按段：%d" % total_duration)

    #添加图形属性
    plt.xlabel('time')
    plt.ylabel('number')
    plt.title('wonderfulTime')
    a = plt.subplot(1, 1, 1)
    plt.ylim=(10, 40000)
    plt.hist(wonderfulTimeArr,40,normed=1,histtype='bar',facecolor='yellowgreen',alpha=0.75)      
    plt.legend()
    plt.show()

plotWonderfulTime()

In [ ]:
def extractMP4toJPG():
    downloaded = os.listdir(baseDir+'videos') # get list of file
    downloaded = [d for d in downloaded if d.endswith('.mp4') or d.endswith('.mkv')]  # keep only .mp4 files
    print 'number of videos downloded are ', len(downloaded) 
    ############################
    fps = 15; # set fps = 0 if you want to extract at original frame rate
    extractframes(sorted(downloaded),fps)
    ###########################    
    saveVidInfo(sorted(downloaded))

# 调用ffmpeg执行mp4提取每帧
def extractframes(vids,fps): # take all .mp4 videos and extract frames using ffmpeg
    for idx,vid in enumerate(vids):
        vidfile = baseDir+'videos/'+vid
        imgdir = baseDir+'images/'+vid.split('.')[0]+'/'
        print idx, vid 
        if not os.path.isdir(imgdir):
            os.mkdir(imgdir)

        imglist = os.listdir(imgdir);
        imglist = [i for i in imglist if i.endswith('.jpg')];

        if len(imglist)<10:
            if fps>0:
                cmd = 'ffmpeg -i {} -qscale:v 5 -r {} {}%05d.jpg'.format(vidfile,fps,imgdir); #-vsync 0
            else:
                cmd = 'ffmpeg -i {} -qscale:v 5 {}%05d.jpg'.format(vidfile,imgdir); #-vsync 0
            # PNG format is very storage heavy so I choose jpg.
            # images will be generated in JPG format with quality scale = 5; you can adjust according to you liking 
            # In appearence it doen't look that deblurred as opposed to default settings by ffmpeg
            # @v 5 images will take alomst 145GB
            #f.write(cmd+'\n')
            os.system(cmd)

In [ ]:
# 加载数据分析常用库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

import paddle.v2 as paddle

In [ ]:
# 方法一：传统Python方法
import cPickle
with open(BROAD_training_filepath+"/100259500.pkl",'rb') as f:
    tmp1 = cPickle.load(f)
print(tmp1.shape)

# # 方法二：利用Pandas读取
# import pandas as pd
# tmp2 = pd.read_pickle(BROAD_training_filepath/xxxx.pkl)
# tmp2

# # 方法三：利用Numpy读取
# import numpy as np
# tmp3 = np.load(BROAD_training_filepath/xxxx.pkl)
# tmp3

In [ ]:
dense_len = 512
rnn_size = 512
num_surround = 50
dim_features = 2048
window_size = 2*num_surround + 1
classnum = 2

reader = paddle.batch(
        mygenerator,
        batch_size=5)

xxx = next(reader())
print(np.array(xxx[0]).shape)

# 0-2047 for np.float32 feature vector and 2048 for 0/1 label vector with length of 101
print(xxx[0][0],xxx[0][2048])

In [ ]:
# 定义卷积神经网络，输入img矩阵
def convolutional_neural_network(img):
    # 第一个卷积-池化层
    conv_pool_1 = paddle.networks.simple_img_conv_pool(
        input=img,
        filter_size=5,
        num_filters=20,
        num_channel=1,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu())
    # 第二个卷积-池化层
    conv_pool_2 = paddle.networks.simple_img_conv_pool(
        input=conv_pool_1,
        filter_size=5,
        num_filters=50,
        num_channel=20,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu())
    # 以softmax为激活函数的全连接输出层，输出层的大小必须为数字的个数10
    predict = paddle.layer.fc(input=conv_pool_2,
                              size=10,
                              act=paddle.activation.Softmax())
    return predict

In [ ]:
paddle.init(use_gpu=False, trainer_count=4)

x = paddle.layer.data(name='x', type=paddle.data_type.dense_vector(13))
y_predict = paddle.layer.fc(input=x,
                                size=1,
                                act=paddle.activation.Linear())
y = paddle.layer.data(name='y', type=paddle.data_type.dense_vector(1))
predict=
cost = paddle.layer.mse_cost(input=y_predict, label=y)

In [ ]:
paddle.init(use_gpu=False, trainer_count=1)

default_std = 1 / math.sqrt(rnn_size) / 3.0
std_default = paddle.attr.Param(initial_std=default_std)
std_0       = paddle.attr.Param(initial_std=0.)
mix_hidden_lr     = 1e-3
lstm_para_attr    = paddle.attr.Param(
    initial_std   = 0.0, 
    learning_rate = 1.0
)
hidden_para_attr = paddle.attr.Param(
    initial_std   = default_std, 
    learning_rate = mix_hidden_lr
)

l_x0 = [paddle.layer.data(name='rnn_layer_%d' % (i), type=paddle.data_type.dense_vector(window_size)) 
            for i in range(dim_features)
]
target = paddle.layer.data(name='label', type=paddle.data_type.integer_value_sequence(classnum))
l_x1 = [
    paddle.layer.embedding(
        size=dense_len, input=x0
    ) for x0 in l_x0
]

hidden_0 = paddle.layer.mixed(
    size=rnn_size,
    bias_attr=std_default,
    input=[
        paddle.layer.full_matrix_projection(
            input=x1, param_attr=std_default
        ) for x1 in l_x1
    ]
)

lstm_0 = paddle.layer.lstmemory(
    input=hidden_0,
    act=paddle.activation.Relu(),
    gate_act=paddle.activation.Sigmoid(),
    state_act=paddle.activation.Sigmoid(),
    bias_attr=std_0,
    param_attr=lstm_para_attr
)

feature_out = paddle.layer.mixed(
    size=classnum,
    bias_attr=std_default,
    input=[
        paddle.layer.full_matrix_projection(
            input=hidden_0, param_attr=hidden_para_attr),
        paddle.layer.full_matrix_projection(
            input=lstm_0, param_attr=lstm_para_attr)
    ]
)

crf_cost = paddle.layer.crf(
    size=classnum,
    input=feature_out,
    label=target,
    param_attr=paddle.attr.Param(
        name='crfw',
        initial_std=default_std,
        learning_rate=mix_hidden_lr
    )
)

crf_dec = paddle.layer.crf_decoding(
   size=classnum,
   input=feature_out,
   label=target,
   param_attr=paddle.attr.Param(name='crfw'))


evaluator.sum(input=crf_dec)

In [ ]:
parameters = paddle.parameters.create(crf_cost)
optimizer = paddle.optimizer.Momentum(
    momentum=0,
    learning_rate=1e-3,
    regularization=paddle.optimizer.L2Regularization(rate=8e-4),
    model_average=paddle.optimizer.ModelAverage(
        average_window=0.5, max_average_window=10000), )

trainer = paddle.trainer.SGD(cost=crf_cost,
                             parameters=parameters,
                             update_equation=optimizer,
                             extra_layers=crf_dec)

l_name = ['rnn_layer_%d' % (i) for i in range(dim_features)] + ['label']
l_idx = range(dim_features + 1)
feeding = dict(zip(l_name, l_idx))

trainer.train(
    reader=reader,
    num_passes=1,
    feeding=feeding
)

In [ ]:
trainer.train(
    reader=paddle.batch(
        paddle.reader.shuffle(
            uci_housing.train(), buf_size=500),
        batch_size=2),
    feeding=feeding,
    event_handler=event_handler,
    num_passes=30)

In [ ]:
# result.json
data={
  "version": "VERSION 1.0",
  "results":
  {
    "164161500": [
      {
        "score": 1.0,
        "segment": [94, 130]
      },
      {
        "score": 0.6, 
        "segment": [150, 196] 
      }
    ]
  }
}
with open('result.json', 'w') as json_file:
    json_file.write(json.dumps(data))

In [ ]:
with open('result.json') as json_file:
    data1 = json.load(json_file)
print(data1)

In [ ]:
run -i '/mnt/BROAD-datasets/video/eval_script/get_detection_performance.py' /mnt/BROAD-datasets/meta.json path_of_result_file --subset validation

In [ ]:
!python2 -m pip install pillow
from PIL import Image
import numpy as np
import os

In [ ]:
def load_image（file):
    im=Image.open(file).convert("L")
    im.resize((28,28),Image.ANTIALIAS)
    im=np.array(im).astype(np.float32).flatten()
    im.im/255.0
    return im

In [ ]:
test_data=[]
cur_dir=os.path.dirname(os.path.realpath(__file__))
test_data.append(load_image(cur_dir+"/image/test.png"))

prob=panddle.infer(
    output_layer=predict,parameters=)
lab=np.argsort(-prob)
print("label of images/test.png is %d" % lab[0][0])